In [1]:
import pathlib

from fytok.modules.Equilibrium import Equilibrium
from fytok.utils.logger import logger
from spdm.view.View import display
WORKSPACE = "/home/salmon/workspace"  # "/ssd01/salmon_work/workspace/"
output_path = pathlib.Path(f"{WORKSPACE}/output/")

2023-11-17 09:41:18,132 [   fytok]     INFO: 
#######################################################################################################################
    ______      _____     _
   / ____/_  __|_   _|__ | | __
  / /_  / / / /  | |/ _ \| |/ /
 / __/ / /_/ /   | | (_) |   <
/_/    \__, /    |_|\___/|_|\_\
      /____/

 Copyright (c) 2021-present Zhi YU (Institute of Plasma Physics Chinese Academy of Sciences)
            
 url: https://gitee.com/openfusion/fytok_tutorial 
      https://github.com/fusion-yun/fytok_tutorial

 version = develop  (spdm=develop extension=0.2.2.dev11+g706f667, imas_version=v3_38_1_dirty)

 Run by salmon at 2023-11-17T09:41:18.132399.
 Job ID: fytok_salmon_surface_18410

#######################################################################################################################



In [2]:
# eq0 = Equilibrium(f"file+GEQdsk://{WORKSPACE}/gacode/neo/tools/input/profile_data/g141459.03890#equilibrium")
eq0 = Equilibrium("file+geqdsk:///home/salmon/workspace/fytok_tutorial/tutorial/data/g900003.00230_ITER_15MA_eqdsk16HR.txt#equilibrium")

2023-11-17 09:41:18,756 [   fytok]    DEBUG: /home/salmon/workspace/fytok/SpDM/python/spdm/utils/sp_export.py:66:sp_load_module: Load module fytok.plugins.equilibrium.fy_eq
2023-11-17 09:41:18,767 [   fytok]    DEBUG: /home/salmon/workspace/fytok/SpDM/python/spdm/utils/sp_export.py:66:sp_load_module: Load module spdm.plugins.data.plugin_geqdsk


In [3]:
eq1d=eq0.time_slice.current.profiles_1d

In [4]:
phi=eq1d.phi 

In [5]:
phi.__array__()


array([  0.        ,   0.30209473,   0.60405784,   0.90596516,
         1.207888  ,   1.50989338,   1.81204431,   2.11439878,
         2.41699452,   2.71983455,   3.02290317,   3.32618366,
         3.6296548 ,   3.93328579,   4.23704351,   4.54090043,
         4.84482984,   5.1488    ,   5.4527767 ,   5.75672684,
         6.06062083,   6.36443462,   6.66814817,   6.97174344,
         7.27520372,   7.57851325,   7.88165805,   8.18462682,
         8.48741081,   8.79000357,   9.09240148,   9.39460437,
         9.69661428,   9.99843398,  10.30006708,  10.60151848,
        10.90279386,  11.20389906,  11.50484068,  11.80562676,
        12.1062664 ,  12.40676919,  12.70714508,  13.00740444,
        13.30755783,  13.60761562,  13.90758877,  14.20748962,
        14.50733156,  14.80712837,  15.10689429,  15.40664419,
        15.70639363,  16.00615888,  16.30595682,  16.60580475,
        16.9057204 ,  17.20572186,  17.50582758,  17.80605635,
        18.1064272 ,  18.40695927,  18.70767223,  19.00